<a href="https://colab.research.google.com/github/SezerBugday/photogrammetry_meshroom_colab_flask/blob/main/Bitirme.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python3 main.py




In [29]:
from pyngrok import ngrok

import os  
import shutil  

from flask_bootstrap import Bootstrap
from flask import Flask, flash, request, redirect, url_for,render_template
from werkzeug.utils import secure_filename

port_no = 5000



In [30]:


app = Flask(__name__)

app._static_folder = '/content/static/'

bootsrapt = Bootstrap(app)
ngrok.set_auth_token("2DJyoS7QMsne6ZsjPBWlik5uBBv_37hajTNHz6NFWXeczXf7A")
public_url =  ngrok.connect(port_no).public_url

@app.route("/")
def HomePage():
  return render_template("index.html")
print(f"To acces the Gloable link please click {public_url}")



if __name__ == "__main__":
    app.run()

To acces the Gloable link please click http://9a1d-35-204-137-218.ngrok.io
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
INFO:werkzeug:127.0.0.1 - - [08/Dec/2022 21:55:53] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/Dec/2022 21:55:54] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [08/Dec/2022 21:56:08] "POST / HTTP/1.1" 405 -


In [41]:
from flask import Flask, flash, request, redirect, url_for, render_template
from werkzeug.utils import secure_filename

TEMPLATE = "/content/templates"
app._static_folder = '/content/static/'

#bootsrapt = Bootstrap(app)
app = Flask(__name__, template_folder=TEMPLATE)

app.config['UPLOAD_FOLDER'] = "/content/kirli"

ngrok.set_auth_token("2DJyoS7QMsne6ZsjPBWlik5uBBv_37hajTNHz6NFWXeczXf7A")
public_url = ngrok.connect(port_no).public_url

print(f"To acces the Gloable link please click {public_url}")


@app.route("/", methods=["GET", "POST"])
def upload_file():
    if request.method == 'POST':
        files = request.files.getlist("image")
        for file in files:
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], file.filename))
            
        os.system("rembg p /content/kirli   /content/temiz")
        os.system("./meshroom/Meshroom-2019.1.0/meshroom_photogrammetry --input ./temiz/ --output ./modelFolder/")
        return render_template("showModel.html")
    return render_template("index.html")


@app.route("/denemeeee")
def Ac():
    return render_template("dene.html")


if __name__ == "__main__":
    app.run()

To acces the Gloable link please click http://6bb5-35-204-137-218.ngrok.io
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
INFO:werkzeug:127.0.0.1 - - [08/Dec/2022 22:17:05] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/Dec/2022 22:17:06] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [08/Dec/2022 22:17:09] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/Dec/2022 22:17:10] "GET /node_modules/xterm/css/xterm.css HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [08/Dec/2022 22:17:10] "GET /static/GoldSword.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [08/Dec/2022 22:17:11] "GET /static/kup.glb HTTP/1.1" 200 -
